In [15]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import urllib.request
import os
from tensorflow.keras.models import load_model
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from langchain_ollama import OllamaLLM
import ipywidgets as widgets
from IPython.display import display, clear_output
import pickle

In [17]:
food_model = load_model('food_classification_model.h5')

In [18]:
classes = ['paella', 'steak', 'bread_pudding', 'waffles', 'french_toast',
       'peking_duck', 'ice_cream', 'hot_dog', 'eggs_benedict', 'lasagna',
       'beef_carpaccio', 'chicken_wings', 'ramen', 'hummus', 'crab_cakes',
       'gnocchi', 'dumplings', 'gyoza', 'clam_chowder', 'beef_tartare',
       'oysters', 'pancakes', 'tiramisu', 'beignets', 'foie_gras',
       'sushi', 'hot_and_sour_soup', 'cannoli', 'strawberry_shortcake',
       'fried_rice', 'pad_thai', 'takoyaki', 'greek_salad',
       'creme_brulee', 'lobster_bisque', 'poutine', 'spaghetti_carbonara',
       'beet_salad', 'baklava', 'sashimi', 'falafel', 'mussels',
       'cheese_plate', 'tacos', 'frozen_yogurt', 'pulled_pork_sandwich',
       'fish_and_chips', 'pho', 'bruschetta', 'onion_rings',
       'chicken_quesadilla', 'deviled_eggs', 'seaweed_salad', 'hamburger',
       'fried_calamari', 'grilled_cheese_sandwich', 'spring_rolls',
       'chocolate_mousse', 'miso_soup', 'nachos', 'grilled_salmon',
       'pork_chop', 'bibimbap', 'escargots', 'red_velvet_cake',
       'cheesecake', 'omelette', 'breakfast_burrito', 'french_fries',
       'macarons', 'ravioli', 'macaroni_and_cheese', 'cup_cakes',
       'shrimp_and_grits', 'baby_back_ribs', 'club_sandwich',
       'filet_mignon', 'samosa', 'tuna_tartare', 'risotto',
       'lobster_roll_sandwich', 'caprese_salad', 'carrot_cake',
       'huevos_rancheros', 'pizza', 'guacamole', 'french_onion_soup',
       'scallops', 'edamame', 'spaghetti_bolognese', 'churros',
       'croque_madame', 'ceviche', 'garlic_bread', 'prime_rib', 'donuts',
       'caesar_salad', 'chicken_curry', 'chocolate_cake', 'panna_cotta',
       'apple_pie']

In [19]:

# Load the saved model
model = load_model('food_classification_model.h5')

# 2. Load class labels
with open('class_labels.pkl', 'rb') as f:
    class_labels = pickle.load(f)
# class_labels = classes

def predict_food(image_path, show_image=True):
    """
    Predict the food type from an image file
    
    Args:
        image_path: path to the image file
        show_image: whether to display the image (default True)
    
    Returns:
        tuple: (predicted_label, confidence_percentage)
    """
    
    # Preprocess the image for the model
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    confidence = np.max(predictions) * 100
    
    return predicted_label, confidence

# Example usage
if __name__ == "__main__":
    input_image = str(sorted(Path("input_image").glob("*"))[0])
    label, confidence = predict_food(input_image)
    print(f"\nPredicted: {label}")
    print(f"Confidence: {confidence:.2f}%")
    print(input_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step

Predicted: edamame
Confidence: 30.97%
input_image\image.png


In [20]:
servings = str(input("\nFor how many people will this be? (1-10): "))
important = str(input("\nIs there anything important that you want to add before the recipe gets generated?\n\n(e.g. Allergies, Prefered ways, difficulty level, duration...)\nIf you don't have anything just press \"Enter\""))

In [21]:
model = OllamaLLM(model="llama3")

if important.strip():
    important = 'Important feature: ' + important
else:
    important = ''
print(important)
if servings.strip() == '' or servings:
    servings = ' is for 2 people'
else:
    try:
        servings_int = int(servings)
        servings_str = f' is for {servings_int} people'
    except ValueError:
        servings_str = ' is for 2 people'
print(servings)

prompt=f"Make a recipe, only including the title, servings{servings}, ingredients and instructions for {label}. {important}"
print(prompt)
result = model.invoke(input=prompt)
print(result)


 is for 2 people
Make a recipe, only including the title, servings is for 2 people, ingredients and instructions for edamame. 
**Sesame Edamame Delight**

Servings: 2

Ingredients:

* 1 cup fresh or frozen edamame
* 2 tablespoons sesame oil
* 1 tablespoon soy sauce
* 1 tablespoon honey
* 1/4 teaspoon red pepper flakes (optional, for some heat)
* Salt and pepper to taste
* Sesame seeds and chopped scallions for garnish (optional)

Instructions:

1. Rinse the edamame under cold water, then pat them dry with a paper towel.
2. In a small bowl, whisk together the sesame oil, soy sauce, honey, and red pepper flakes (if using).
3. Add the edamame to the bowl and toss them to coat evenly with the marinade.
4. Cover the bowl with plastic wrap and refrigerate for at least 30 minutes or up to 2 hours to allow the flavors to meld.
5. Preheat a grill or grill pan over medium-high heat. Remove the edamame from the refrigerator and thread them onto skewers or place them in a single layer on a baking